In [1]:
from neo4j import GraphDatabase
import pandas as pd

# 接続、初期化

In [2]:
uri = 'bolt://neo4j:7687'
user = 'neo4j'
password = 'neo4j'
driver = GraphDatabase.driver(uri, auth=(user, password))

In [3]:
def clear_db(tx):
    '''
    ノードと関係性を全て削除する
    '''
    tx.run('MATCH (n) DETACH DELETE n')

def get_constrs(tx):
    '''
    制約名リストを取得する
    '''
    constrs = tx.run('CALL db.constraints')
    constrs_name = []
    for constr in constrs:
        constrs_name.append(constr.value('name'))
    return constrs_name
    
def drop_constrs(tx, constrs):
    '''
    制約を削除する
    '''
    if not isinstance(constrs, list):
        constrs = [constrs]
    for constr in constrs:
        tx.run('DROP CONSTRAINT ' + constr)

In [4]:
# ノード削除
with driver.session() as session:
    session.write_transaction(clear_db)
# 制約削除
with driver.session() as session:
    constrs = session.read_transaction(get_constrs)
    session.write_transaction(drop_constrs, constrs)

# 作業順序への適用

+ パターン毎の作業順序をグラフDB化
+ 工程マスタ、作業マスタ、作業順序を兼ねられる？？
    + 「ある工程の全作業」「ある作業が属する工程」など取得する関数を作ればOK
    + 外部キー制約等は付けられないからダメ？？

## neo4jへのデータ登録

In [5]:
WORK = '作業'
KOUTEI = '工程'
PATTERN = 'パターン'
COLNAME_ORDER_ID = '作業順序ID'
COLNAME_PRETASK = '前作業'
COLNAME_POSTTASK = '後作業'
COLNAME_PRETASK_ORIGIN = '前作業原点'
COLNAME_POSTTASK_ORIGIN = '後作業原点'
COLNAME_OFFSET_TIME = '時間差'
COLNAME_CONSTR_TYPE = '制約種類'

In [6]:
path_sagyou_master = './data/sagyou_master_demo.xlsx'
path_sagyou_order = './data/sagyou_order_demo.xlsx'

df_sagyou_master = pd.read_excel(path_sagyou_master)
df_sagyou_order = pd.read_excel(path_sagyou_order)

In [7]:
def add_unique_sagyou(tx):
    '''
    作業ノードの「作業」属性にUNIQUE制約付与
    '''
    tx.run('CREATE CONSTRAINT ON (w:Work) ASSERT w.work IS UNIQUE')

# def add_unique_sagyou_order(tx):
#     '''
#     作業順序リレーションの「パターン」「作業順序ID」にUNIQUE制約付与
#     （有料版でないと効かない？）
#     '''
#     tx.run('CREATE CONSTRAINT ON (o:ORDER) ASSERT (o.pattern, o.order_id) IS UNIQUE')

def add_sagyou(tx, df):
    '''
    作業ノードの登録
    各行につき1ノードとし、列「作業」「工程」を属性として追加
    '''
    for _, ds in df.iterrows():
        tx.run(f'CREATE (:Work:{ds[KOUTEI]} {{work: $work}})',
               work=ds[WORK])

def add_sagyou_order(tx, df):
    '''
    作業順序リレーションを登録
    各行につき1リレーションとし、各列（前作業、後作業以外）を属性として追加
    '''
    for _, ds in df.iterrows():
        cypher = f'''
            MATCH
                (a:Work{{work:$pretask}}),(b:Work{{work:$posttask}})
            CREATE
                (a)-[:{encode_pattern(ds[PATTERN])}{{
                    order_id: $order_id,
                    pretask_origin: $pretask_origin,
                    posttask_origin: $posttask_origin,
                    offset_time: $offset_time,
                    constr_type: $constr_type
                }}]->(b);
        '''
        tx.run(
            cypher,
            pretask=ds[COLNAME_PRETASK],
            posttask=ds[COLNAME_POSTTASK],
            order_id=ds[COLNAME_ORDER_ID],
            pretask_origin=ds[COLNAME_PRETASK_ORIGIN],
            posttask_origin=ds[COLNAME_POSTTASK_ORIGIN],
            offset_time=ds[COLNAME_POSTTASK_ORIGIN],
            constr_type=ds[COLNAME_CONSTR_TYPE]
        )

def encode_pattern(pattern):
    '''
    パターン名をneo4j用のものに変換する
    例：「A-1」→「A_1」
    '''
    return pattern.replace('-', '_')

def decode_pattern(pattern):
    '''
    パターン名をneo4j用のものから変換する
    例：「A_1」→「A-1」
    '''
    return pattern.replace('_', '-')

In [8]:
# 制約追加
with driver.session() as session:
    session.write_transaction(add_unique_sagyou)
#     session.write_transaction(add_unique_sagyou_order)

In [9]:
# ノード、リレーション追加
with driver.session() as session:
    session.write_transaction(add_sagyou, df_sagyou_master)
    session.write_transaction(add_sagyou_order, df_sagyou_order)

→neo4jのGUIでパターンを絞っても、表示される関係性にフィルタがかからない